In [1]:
import tensorflow as tf
import numpy as np

In [2]:
a = tf.constant(3.0, dtype = tf.float32)
b = tf.constant(4.0)
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


Statements above define the computation graph. They do not execute

In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

Open tensorboard by entering the following shell command

tensorboard --logdir .

Open in chrome

localhost:6006

In [7]:
# Run session
sess = tf.Session()
print(sess.run(total))

7.0


In [8]:
sess.run({'ab': (a, b), 'total': total})

{'ab': (3.0, 4.0), 'total': 7.0}

In [11]:
# Tensors have a single value for each call to tf.Session.run
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2

# Different for each call
print(sess.run(vec))
print(sess.run(vec))

# Consistent within a single call
print(sess.run((out1, out2)))

[0.9093877  0.75254667 0.46395898]
[0.41055417 0.89399695 0.7927828 ]
(array([1.8888285, 1.7362406, 1.8416642], dtype=float32), array([2.8888285, 2.7362406, 2.8416643], dtype=float32))


In [19]:
# Data can be fed into a graph using placeholders

x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

# Pass in values with feed_dict
print(sess.run(z, feed_dict ={x: 5, y: 6}))

print(sess.run(z, feed_dict ={x: [5, 6], y: 6}))

# Can overwrite values in graph
print(sess.run(z, feed_dict ={x: [5, 6], y: 6, z: -1}))

11.0
[11. 12.]
-1.0


In [27]:
# Datasets are preferred method to pass data into graph

my_data = [
    [0, 1],
    [2, 3],
    [4, 5]    
]

slices = tf.data.Dataset.from_tensor_slices(my_data)

x = slices.make_one_shot_iterator().get_next()

out = x*5 + 3


In [28]:
while True:
    try:
        print(sess.run(out))
    except tf.errors.OutOfRangeError:
        break

[3 8]
[13 18]
[23 28]


In [42]:
## Defining a model with layers

x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

# Layers must be initialized
init = tf.global_variables_initializer()
sess.run(init)

sess.run(y, feed_dict={x: [[4, 5, 6], [3, 2, 1]]})

array([[-0.7834084],
       [-1.4206883]], dtype=float32)

In [47]:
"""
Defining a model with shortcut layers

Cons:
Cannot resue layers
Cannot access tf.layers.Layer object
May be more difficult to debug
"""

x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

# Layers must be initialized
init = tf.global_variables_initializer()
sess.run(init)

sess.run(y, feed_dict={x: [[4, 5, 6], [3, 2, 1]]})

array([[-5.6478987],
       [-1.8815993]], dtype=float32)

In [66]:
x = tf.constant([[1, 2], [2, 3], [3, 4], [4, 5]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
         
print("Loss Values")
for i in range(10):
  _, loss_value = sess.run((train, loss))
  print("Iteration %i: %.3f" %(i, loss_value))

print("\nPredicted values")
print(sess.run(y_pred))

print("\nKernel Weights")
print(sess.run(linear_model.weights[0]))

print("\nBias Weight")
print(sess.run(linear_model.weights[1]))

Loss Values
Iteration 0: 3.739
Iteration 1: 1.833
Iteration 2: 1.224
Iteration 3: 1.026
Iteration 4: 0.959
Iteration 5: 0.933
Iteration 6: 0.921
Iteration 7: 0.912
Iteration 8: 0.905
Iteration 9: 0.899

Predicted values
[[-1.5047321]
 [-1.704733 ]
 [-1.904734 ]
 [-2.1047351]]

Kernel Weights
[[ 1.0959226]
 [-1.2959236]]

Bias Weight
[-0.00880751]


In [56]:
linear_model.weights

[<tf.Variable 'dense_16/kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'dense_16/bias:0' shape=(1,) dtype=float32_ref>]